In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import time

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# Transformação do dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Converte para 3 canais
    transforms.Resize((128, 128)),  # Redimensiona para 128x128
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
# Configurações iniciais
batch_size = 64
learning_rate = 0.001
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Carregamento do dataset Fashion-MNIST
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26421880/26421880 [00:02<00:00, 12448707.80it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 269609.88it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4966198.80it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 23267539.86it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [6]:
# Função de treinamento
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

In [7]:
# Função de avaliação
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [8]:
# Função principal para treinar e avaliar os modelos
def train_and_evaluate(model_name, pre_training):
    # Carrega o modelo do timm e ajusta para 10 classes (Fashion-MNIST)
    model = timm.create_model(model_name, pretrained=pre_training, num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"\nTreinando o modelo: {model_name}")
    start_time = time.time()

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
        test_accuracy = evaluate(model, test_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    end_time = time.time()
    print(f"Tempo total de treinamento para {model_name}: {(end_time - start_time) / 60:.2f} minutos\n")

In [9]:
pre_trained_model = 'tf_efficientnetv2_s.in21k'  # EfficientNetV2

for i in range (1, 6):
    print(f"=====> Iter = '{i}, Model = {pre_trained_model}")
    train_and_evaluate(pre_trained_model, pre_training=True)
    print(f"=====> FINALIZADO | Iter = {i} {pre_trained_model}\n\n")

#train_and_evaluate(raw_model, pre_training=False)

#print(f"FINALIZADO {raw_model}\n")

=====> Iter = '1, Model = tf_efficientnetv2_s.in21k


model.safetensors:   0%|          | 0.00/193M [00:00<?, ?B/s]


Treinando o modelo: tf_efficientnetv2_s.in21k
Epoch [1/10], Loss: 0.4096, Test Accuracy: 0.9147
Epoch [2/10], Loss: 0.2388, Test Accuracy: 0.9233
Epoch [3/10], Loss: 0.1820, Test Accuracy: 0.9313
Epoch [4/10], Loss: 0.1428, Test Accuracy: 0.9289
Epoch [5/10], Loss: 0.1274, Test Accuracy: 0.9383
Epoch [6/10], Loss: 0.1276, Test Accuracy: 0.9333
Epoch [7/10], Loss: 0.1911, Test Accuracy: 0.9288
Epoch [8/10], Loss: 0.1369, Test Accuracy: 0.9292
Epoch [9/10], Loss: 0.1040, Test Accuracy: 0.9325
Epoch [10/10], Loss: 0.0957, Test Accuracy: 0.9384
Tempo total de treinamento para tf_efficientnetv2_s.in21k: 46.37 minutos

=====> FINALIZADO | Iter = 1 tf_efficientnetv2_s.in21k


=====> Iter = '2, Model = tf_efficientnetv2_s.in21k

Treinando o modelo: tf_efficientnetv2_s.in21k
Epoch [1/10], Loss: 0.3369, Test Accuracy: 0.9320
Epoch [2/10], Loss: 0.1595, Test Accuracy: 0.9376
Epoch [3/10], Loss: 0.2200, Test Accuracy: 0.9166
Epoch [4/10], Loss: 0.1898, Test Accuracy: 0.9343
Epoch [5/10], Loss: 0.